## Decision Tree



In [ ]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.left = left
        self.right = right
        self.feature = feature
        self.value = value
        self.threshold = threshold


class DecisionTree:
    def __init__(self, criteria = 'gini', max_depth=10, min_samples=1):
        self.criteria = criteria
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None

    def fit(self, X, y):
        self.root = self.build_tree(X, y)
        return self
    
    def build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))

        if (depth >= self.max_depth) or (num_classes == 1) or (num_samples < self.min_samples):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)

        best_feature, best_threshold = self.best_split(X, y)

        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)

        X_l, y_l, X_r, y_r = self.split_data(X, y, best_feature, best_threshold)

        # Recursively build left and right subtrees
        left_child = self.build_tree(X_l, y_l, depth + 1)
        right_child = self.build_tree(X_r, y_r, depth + 1)

        return Node(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)

    def best_split(self, X, y):
        best_feature = None
        best_gain = -1
        best_threshold = None
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                X_l, y_l, X_r, y_r = self.split_data(X, y, feature_index, threshold)
                
                if np.sum(y_l) == 0 or np.sum(y_r) == 0:
                    continue

                if self.information_gain(y, y_l, y_r) > best_gain:
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold
    
    def information_gain(self, parent, left_child, right_child):
        n = len(parent)
        n_left = len(left_child)
        n_right = len(right_child)
        if (n_left == 0) or (n_right) == 0:
            return 0

        parent_impurity = self.impurity(parent)
        child_impurity = (n_left / n) * self.impurity(left_child) + (n_right / n) * self.impurity(right_child)
        return parent_impurity - child_impurity

    def impurity(self, y):
        if self.criteria == 'gini':
            return self.gini_value(y)
        elif self.criteria == 'entropy':
            return self.entropy(y)
        else:
            raise NotImplementedError

    def gini_value(self, y):
        _, counts = np.unique(y, return_counts = True)
        probs = counts/np.sum(counts)
        return 1 - np.sum(probs ** 2)

    def entropy(self, y):
        _, counts = np.unique(y, return_counts = True)
        probs = counts/np.sum(counts)
        return - np.sum(probs * np.log2(probs + 1e-9))

    def split_data(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    

    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]



    def predict(self, X):
        return np.array([self.traverse_tree(x, self.root) for x in X])

    def traverse_tree(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        else:
            return self.traverse_tree(x, node.right)

    def print_tree(self, node=None, depth=0):
        """Print a text representation of the tree."""
        if node is None:
            node = self.root
        
        if node.value is not None:
            print(f"{'  ' * depth}Leaf: class={node.value}")
        else:
            print(f"{'  ' * depth}Feature {node.feature} <= {node.threshold:.3f}")
            print(f"{'  ' * depth}Left:")
            self.print_tree(node.left, depth + 1)
            print(f"{'  ' * depth}Right:")
            self.print_tree(node.right, depth + 1)
        



In [49]:
from sklearn.datasets import load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print("=" * 60)
print("Decision Tree from Scratch - Example Usage")
print("=" * 60)

# Example 1: Iris dataset
print("\n1. Testing on Iris Dataset")
print("-" * 60)
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train with Gini criterion
dt_gini = DecisionTree(max_depth=5, criteria='gini')
dt_gini.fit(X_train, y_train)
y_pred_gini = dt_gini.predict(X_test)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
print(f"Accuracy (Gini): {accuracy_gini:.4f}")

# Train with Entropy criterion
dt_entropy = DecisionTree(max_depth=5, criteria='entropy')
dt_entropy.fit(X_train, y_train)
y_pred_entropy = dt_entropy.predict(X_test)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
print(f"Accuracy (Entropy): {accuracy_entropy:.4f}")

# Example 2: Simple custom dataset
print("\n2. Testing on Simple Custom Dataset")
print("-" * 60)
X_simple = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]])
y_simple = np.array([0, 0, 0, 1, 1, 1])

dt_simple = DecisionTree(max_depth=3)
dt_simple.fit(X_simple, y_simple)

print("\nTree Structure:")
dt_simple.print_tree()

print("\nPredictions:")
test_samples = np.array([[1.5, 2.5], [5.5, 6.5]])
predictions = dt_simple.predict(test_samples)
for sample, pred in zip(test_samples, predictions):
    print(f"Sample {sample} -> Class {pred}")

# Example 3: Larger synthetic dataset
print("\n3. Testing on Synthetic Dataset")
print("-" * 60)
X_synthetic, y_synthetic = make_classification(
    n_samples=500, n_features=10, n_informative=5, 
    n_redundant=2, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X_synthetic, y_synthetic, test_size=0.2, random_state=42
)

dt_synthetic = DecisionTree(max_depth=10, min_samples=5)
dt_synthetic.fit(X_train, y_train)
y_pred_synthetic = dt_synthetic.predict(X_test)
accuracy_synthetic = accuracy_score(y_test, y_pred_synthetic)
print(f"Accuracy: {accuracy_synthetic:.4f}")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

print("\n" + "=" * 60)
print("All tests completed successfully!")
print("=" * 60)

Decision Tree from Scratch - Example Usage

1. Testing on Iris Dataset
------------------------------------------------------------
Accuracy (Gini): 0.5333
Accuracy (Entropy): 0.5333

2. Testing on Simple Custom Dataset
------------------------------------------------------------

Tree Structure:
Feature 1 <= 6.000
Left:
  Feature 1 <= 5.000
  Left:
    Leaf: class=0
  Right:
    Leaf: class=1
Right:
  Leaf: class=1

Predictions:
Sample [1.5 2.5] -> Class 0
Sample [5.5 6.5] -> Class 1

3. Testing on Synthetic Dataset
------------------------------------------------------------
Accuracy: 0.5000
Training samples: 400
Test samples: 100

All tests completed successfully!
